In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator

load_dotenv()
llm = ChatOpenAI(model="qwen-plus")

In [4]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [5]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables="query",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [6]:
# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | llm
output = prompt_and_model.invoke({"query": "Tell me a joke."})
output.pretty_print()

================================== Ai Message ==================================

{"setup": "Why don't skeletons fight each other?", "punchline": "Because they don't have the guts."}


In [7]:
# And a query intended to prompt a language model to populate the data structure.
chain = prompt | llm | parser
output = chain.invoke({"query": "Tell me a joke."})
output

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

In [8]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | llm | json_parser

In [9]:
list(json_chain.stream({"question": "Who invented the microscope?"}))

[{},
 {'answer': ''},
 {'answer': 'The microscope'},
 {'answer': 'The microscope was'},
 {'answer': 'The microscope was invented by Hans L'},
 {'answer': 'The microscope was invented by Hans Lippershey, though Zach'},
 {'answer': 'The microscope was invented by Hans Lippershey, though Zacharias Janssen'},
 {'answer': 'The microscope was invented by Hans Lippershey, though Zacharias Janssen and Hans Janssen are'},
 {'answer': 'The microscope was invented by Hans Lippershey, though Zacharias Janssen and Hans Janssen are often credited with creating'},
 {'answer': 'The microscope was invented by Hans Lippershey, though Zacharias Janssen and Hans Janssen are often credited with creating an early version of the compound'},
 {'answer': 'The microscope was invented by Hans Lippershey, though Zacharias Janssen and Hans Janssen are often credited with creating an early version of the compound microscope in the late 1'},
 {'answer': 'The microscope was invented by Hans Lippershey, though Zachari

In [43]:
list(chain.stream({"query": "Tell me a joke."}))

[Joke(setup="Why don't scientists trust atoms?", punchline=''),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')]

In [17]:
json_parser.invoke('{"query": "Tell me a joke."}')

{'query': 'Tell me a joke.'}